In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial import distance
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

# Reading the images
img1 = cv2.imread("/Users/macbook/Downloads/Plaksha_Faculty.jpg")
img2 = cv2.imread("/Users/macbook/Downloads/Dr_Shashi_Tharoor.jpg")

# Convert to grayscale
gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Loading the required haar-cascade xml classifier file
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Applying face detection method
faces_rect1 = face_cascade.detectMultiScale(gray_img1, 1.05, 4, minSize=(25,25), maxSize=(50,50))
faces_rect2 = face_cascade.detectMultiScale(gray_img2, 1.05, 4, minSize=(25,25), maxSize=(50,50))

# Combine detected faces
faces_rect = np.vstack((faces_rect1, faces_rect2)) if len(faces_rect1) and len(faces_rect2) else faces_rect1 if len(faces_rect1) else faces_rect2

# Define text parameters
text = "Detected Face"
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (0, 0, 255)  # Red
font_thickness = 1

# Draw rectangles and add text
for (x, y, w, h) in faces_rect:
    cv2.rectangle(img1, (x, y), (x+w, y+h), (0, 0, 255), 2)
    cv2.putText(img1, text, (x, y-10), font, font_scale, font_color, font_thickness)

# Display the image
cv2.imshow(f"Total number of faces detected: {len(faces_rect)}", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Convert to HSV
img_hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
img_hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
hue_saturation = []
face_images = []

for (x, y, w, h) in faces_rect:
    face = img_hsv1[y:y + h, x:x + w] if y + h <= img_hsv1.shape[0] else img_hsv2[y:y + h, x:x + w]
    hue = np.mean(face[:, :, 0])
    saturation = np.mean(face[:, :, 1])
    hue_saturation.append((hue, saturation))
    face_images.append(face)

hue_saturation = np.array(hue_saturation)

# Perform k-Means clustering
kmeans = KMeans(n_clusters=2, random_state=42).fit(hue_saturation)

# Plot clustered faces
fig, ax = plt.subplots(figsize=(12, 6))
for i, (x, y, w, h) in enumerate(faces_rect):
    im = OffsetImage(cv2.cvtColor(cv2.resize(face_images[i], (20, 20)), cv2.COLOR_HSV2RGB))
    ab = AnnotationBbox(im, (hue_saturation[i, 0], hue_saturation[i, 1]), frameon=False, pad=0)
    ax.add_artist(ab)
    plt.plot(hue_saturation[i, 0], hue_saturation[i, 1])

plt.xlabel("Hue")
plt.ylabel("Saturation")
plt.title("Clustered Faces")
plt.grid()
plt.show()

# Scatter plot for clusters
fig, ax = plt.subplots(figsize=(12, 6))
cluster_0_points = hue_saturation[kmeans.labels_ == 0]
cluster_1_points = hue_saturation[kmeans.labels_ == 1]
plt.scatter(cluster_0_points[:, 0], cluster_0_points[:, 1], color='green', label='Cluster 0')
plt.scatter(cluster_1_points[:, 0], cluster_1_points[:, 1], color='blue', label='Cluster 1')

# Plot centroids
centroid_0 = kmeans.cluster_centers_[0]
centroid_1 = kmeans.cluster_centers_[1]
plt.scatter(centroid_0[0], centroid_0[1], marker='x', color='black', s=100, label='Centroid 0')
plt.scatter(centroid_1[0], centroid_1[1], marker='x', color='red', s=100, label='Centroid 1')

plt.xlabel("Hue")
plt.ylabel("Saturation")
plt.title("Clustered Faces with Centroids")
plt.legend()
plt.grid()
plt.show()
